In [48]:
from numba import cuda
from numba import *
import numpy as np
import math

In [49]:
# Device Properties to get execution config
# Get the current device
device = cuda.get_current_device()

# Access device properties
warp_size = device.WARP_SIZE
multi_processor_count = device.MULTIPROCESSOR_COUNT

# print(warp_size)
# print(multi_processor_count)
tpb = warp_size                     # threads per block
nb = multi_processor_count * 32     # number of blocks
print(tpb)
print(nb)

32
512


In [50]:
tpb2d = tpb

In [51]:
@cuda.jit
def transformv2(A:np.ndarray, b: np.ndarray, X: np.ndarray, Y: np.ndarray):
    # tpb2d, _ = cuda.blockDim(2)

    sX = cuda.shared.array(shape=(tpb2d,tpb2d), dtype=float64)
    sA = cuda.shared.array(shape=(tpb2d,tpb2d), dtype=float64)

    x, y = cuda.grid(2)
    # stridex, stridey = cuda.gridsize(2)

    tx, ty = cuda.threadIdx.x, cuda.threadIdx.y 

    bpg = cuda.gridDim.x    # blocks per grid, aka grid Dim

    
    # for i in range(idx,Y.shape[0],stridex):
    #     for j in range(idy, Y.shape[1],stridey):
    temp = float64(0.)
    for i in range(bpg):
        
        # Preload chunks of data into shared memory
        sX[tx,ty] = 0 
        sA[tx,ty] = 0
        if y < X.shape[0] and (tx + i * tpb2d) < X.shape[1]:
            sX[ty,tx] = X[y,tx + i * tpb2d]
        if x < A.shape[1] and (ty + i * tpb2d) < A.shape[0]:
            sA[ty, tx] = A[ty + i * tpb2d, x]

        # wait till loading complete
        cuda.syncthreads()

        # Do partial row * col
        for j in range(tpb2d):
            temp += sX[ty, j] * sA[j, tx]

        # Sync again
        cuda.syncthreads()

    # Put result back in
    if y < Y.shape[0] and x < Y.shape[1]:
        Y[y,x] = temp + b[0,x]

In [52]:
# Test Setup
N = 20
A = np.random.random((5,5)).astype(np.float32)
X = np.random.random((N,5)).astype(np.float32)
b = np.random.random((1,5)).astype(np.float32)
Y = np.zeros_like(X).astype(np.float32)

In [53]:
tpb2d_ = (tpb,tpb)
# nb2d_ = (math.ceil(Y.shape[0]/tpb)(Y.s),math.cel)
nb2d_ = (math.ceil(Y.shape[0] / tpb), math.ceil(Y.shape[1]/tpb))

# Shared memory size
# shared_mem_size = 2 * tpb * tpb * np.dtype(np.float64).itemsize

print(tpb2d_)
print(nb2d_)
# print(shared_mem_size)

(32, 32)
(1, 1)


In [54]:
truth = X@A  + np.tile(b,(N,1))
btruth = X@A + b

# tranformsl(A,b,X,Y)
dA = cuda.to_device(A)
dX = cuda.to_device(X)
db = cuda.to_device(b)
dY = cuda.to_device(Y)
# tranformv1[nb,tpb](dA,db,dX,dY)
transformv2[nb2d_,tpb2d_](dA,db,dX,dY)
Y = dY.copy_to_host()

/home/harris/miniconda3/envs/vmcmc/lib/python3.10/site-packages/numba/cuda/dispatcher.py:536: NumbaPerformanceWarning: Grid size 1 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


In [55]:
Y - truth

array([[ 0.0000000e+00,  0.0000000e+00, -1.1920929e-07,  0.0000000e+00,
         0.0000000e+00],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
        -1.1920929e-07],
       [ 5.9604645e-08, -1.1920929e-07,  0.0000000e+00,  0.0000000e+00,
         0.0000000e+00],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
         0.0000000e+00],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
         0.0000000e+00],
       [ 0.0000000e+00,  1.1920929e-07,  0.0000000e+00,  0.0000000e+00,
         0.0000000e+00],
       [ 0.0000000e+00,  1.1920929e-07, -1.1920929e-07,  0.0000000e+00,
         0.0000000e+00],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
         0.0000000e+00],
       [ 0.0000000e+00,  0.0000000e+00,  1.1920929e-07,  0.0000000e+00,
         0.0000000e+00],
       [ 0.0000000e+00,  0.0000000e+00, -1.1920929e-07, -1.1920929e-07,
         5.9604645e-08],
       [ 2.9802322e-08,  0.000